### Spoken Language Processing
Homework 1
1. Ссылка на занятие: https://www.twitch.tv/deeplearningschool
2. Материалы: https://vk.cc/bVLCOC https://vk.cc/bVLD3Z

В этом задании предлагается обучить классификатор класса возраста по голосу (пример с тем, как это можно сделать для пола см. в семинаре)

P.S. не забудьте, что если то вы работает в Colab, то вы можете поменять среду выполнения на GPU/TPU!

Вопросы по заданию/материалам: @Nestyme

In [1]:
!pip3 install timit-utils==0.9.0

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5890 sha256=b23ea0e23313d68a01716789c4ece63c1814ddf4ffa5aae0e41cdecb5010117f
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [2]:
!pip3 install torchaudio

     |████████████████████████████████| 7.6MB 3.6MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101


In [3]:
! wget https://ndownloader.figshare.com/files/10256148 

--2020-12-27 14:06:44--  https://ndownloader.figshare.com/files/10256148
Resolving ndownloader.figshare.com (ndownloader.figshare.com)... 34.255.82.212, 52.16.75.60, 3.248.141.196, ...
Connecting to ndownloader.figshare.com (ndownloader.figshare.com)|34.255.82.212|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/10256148/TIMIT.zip [following]
--2020-12-27 14:06:44--  https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/10256148/TIMIT.zip
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.40.131
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.40.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 440207227 (420M) [binary/octet-stream]
Saving to: ‘10256148’

10256148            100%[===================>] 419.81M  30.1MB/s    in 15s     

2020-12-27 14:06:59 (28.5 MB/s) - ‘10256148’ saved [440207227/440207227]



In [4]:
!unzip -q 10256148

In [5]:
import timit_utils as tu
import os
import librosa
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

import IPython
_TIMIT_PATH = 'data/lisa/data/timit/raw/TIMIT'

## Задание 1
Загрузите данные для обучения. Для этого:
1. Скачайте датасет TIMIT (см семинар)
2. Соберите пары "голос"  — "класс возраста" также, как на семинаре собирались пары "голос"  — "пол". Аудиодорожки сконвертируйте в мелспектрограммы при помощи `torchaudio либо` `librosa`

P.S. вы можете использовать свою реализацию, а можете предложенную (см следующие ячейки)

In [6]:
import timit_utils as tu
import os
import librosa
import numpy as np
from tqdm import tqdm
import torch as t


class timit_dataloader:
    def __init__(self, data_path=_TIMIT_PATH, train_mode=True, age_mode=True):
        self.doc_file_path = os.path.join(data_path, 'DOC', 'SPKRINFO.TXT')
        self.corpus = tu.Corpus(data_path)
        with open(self.doc_file_path) as f:
            self.id_age_dict = dict(
                [(tmp.split(' ')[0], 86 - int(tmp.split('  ')[5].split('/')[-1].replace('??', '50'))) \
                 for tmp in f.readlines()[39:]])
        if train_mode:
            self.trainset = self.create_dataset('train', age_mode=age_mode)
            self.validset = self.create_dataset('valid', age_mode=age_mode)
        self.testset = self.create_dataset('test', age_mode=age_mode)

    def return_age(self, id):
        return self.id_age_dict[id]

    def return_data(self):
        return self.trainset, self.validset, self.testset

    def return_test(self):
        return self.testset

    def create_dataset(self, mode, age_mode=False):
        global people
        assert mode in ['train', 'valid', 'test']
        if mode == 'train':
            people = [self.corpus.train.person_by_index(i) for i in range(350)]
        if mode == 'valid':
            people = [self.corpus.train.person_by_index(i) for i in range(350, 400)]
        if mode == 'test':
            people = [self.corpus.test.person_by_index(i) for i in range(150)]
        spectrograms_and_targets = []
        for person in tqdm(people):
              try:
                  target = self.return_age(person.name)
                  for i in range(len(person.sentences)):
                      spectrograms_and_targets.append(
                          self.preprocess_sample(person.sentence_by_index(i).raw_audio, target, age_mode=True))
              except:
                  print(person.name, target)

        X, y = map(np.stack, zip(*spectrograms_and_targets))
        X = X.transpose([0, 2, 1])  # to [batch, time, channels]
        return X, y

    @staticmethod
    def spec_to_image(spec, eps=1e-6):
        mean = spec.mean()
        std = spec.std()
        spec_norm = (spec - mean) / (std + eps)
        spec_min, spec_max = spec_norm.min(), spec_norm.max()
        spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
        spec_scaled = spec_scaled.astype(np.uint8)
        return spec_scaled

    @staticmethod
    def clasterize_by_age(age):
        if age <= 25:
            return 0
        if age > 25 and age <=35:
            return 1
        if age >35:
          return 2
        

    def preprocess_sample(self, amplitudes, target, age_mode=False, sr=16000, max_length=150):
        spectrogram = librosa.feature.melspectrogram(amplitudes, sr=sr, n_mels=256, fmin=1, fmax=8192)[:, :max_length]
        spectrogram = np.pad(spectrogram, [[0, 0], [0, max(0, max_length - spectrogram.shape[1])]], mode='constant')
        target = self.clasterize_by_age(target)
        return self.spec_to_image(np.float32(spectrogram)), target

    def preprocess_sample_inference(self, amplitudes, sr=16000, max_length=150, device='cpu'):
        spectrogram = librosa.feature.melspectrogram(amplitudes, sr=sr, n_mels=256, fmin=1, fmax=8192)[:, :max_length]
        spectrogram = np.pad(spectrogram, [[0, 0], [0, max(0, max_length - spectrogram.shape[1])]], mode='constant')
        spectrogram = np.array([self.spec_to_image(np.float32(spectrogram))]).transpose([0, 2, 1])

        return t.tensor(spectrogram, dtype=t.float).to(device, non_blocking=True)


class dataloader:
    def __init__(self, spectrograms, targets):
        self.data = list(zip(spectrograms, targets))

    def next_batch(self, batch_size, device):
        indices = np.random.randint(len(self.data), size=batch_size)

        input = [self.data[i] for i in indices]

        source = [line[0] for line in input]
        target = [line[1] for line in input]

        return self.torch_batch(source, target, device)

    @staticmethod
    def torch_batch(source, target, device):
        #print(source)
        #print(target)
        return tuple(
            [
                t.tensor(val, dtype=t.float).to(device, non_blocking=True)
                for val in [source, target]
            ]
        )

    @staticmethod
    def padd_sequences(lines, pad_token=0):
        lengths = [len(line) for line in lines]
        max_length = max(lengths)

        return np.array(
            [
                line + [pad_token] * (max_length - lengths[i])
                for i, line in enumerate(lines)
            ]
        )

Простая сверточная сеть, ее можно дотюнить или поменять по желанию

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim.adamw

class Model(nn.Module):
    def __init__(self, window_sizes=(3, 4, 5, 6)):
        super(Model, self).__init__()

        self.convs = nn.ModuleList([
            nn.Conv2d(1, 256, [window_size, 256], padding=(window_size - 1, 0))
            for window_size in window_sizes
        ])

        self.fc1 = nn.Linear(256 * len(window_sizes), 514)
        self.fc2 = nn.Linear(514,256)
        self.fc3 = nn.Linear(256,3)

    def forward(self, x):
        # Apply a convolution + max pool layer for each window size
        x = torch.unsqueeze(x, 1)  # [B, C, T, E] Add a channel dim.
        xs = []
        for conv in self.convs:
            x2 = F.relu(conv(x))  # [B, F, T, 1]
            x2 = torch.squeeze(x2, -1)  # [B, F, T]
            x2 = F.max_pool1d(x2, x2.size(2))  # [B, F, 1]
            xs.append(x2)
        x = torch.cat(xs, 2)  # [B, F, window]

        # FC
        x = x.view(x.size(0), -1)  # [B, F * window]
        logits1 = self.fc1(x)  # [B, class]
        logits2=self.fc2(logits1)
        probs=self.fc3(logits2)
        return probs

    def loss(self, probs, targets):
        return nn.CrossEntropyLoss()(probs, targets)

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'using {device} mode')
patience = 500
best_loss = 1000
cnt = 0

model = Model()
if device == torch.device('cuda'):
    model.cuda()
else:
    model.cpu()
model.train()

using cpu mode


Model(
  (convs): ModuleList(
    (0): Conv2d(1, 256, kernel_size=[3, 256], stride=(1, 1), padding=(2, 0))
    (1): Conv2d(1, 256, kernel_size=[4, 256], stride=(1, 1), padding=(3, 0))
    (2): Conv2d(1, 256, kernel_size=[5, 256], stride=(1, 1), padding=(4, 0))
    (3): Conv2d(1, 256, kernel_size=[6, 256], stride=(1, 1), padding=(5, 0))
  )
  (fc1): Linear(in_features=1024, out_features=514, bias=True)
  (fc2): Linear(in_features=514, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=3, bias=True)
)

In [17]:
_timit_dataloader = timit_dataloader()
train, valid, test = _timit_dataloader.return_data()

trainset = dataloader(*train)
validset = dataloader(*valid)
testset = dataloader(*test)
BATCH_SIZE = 64

optimizer = torch.optim.AdamW(
    [p for p in model.parameters() if p.requires_grad], eps=1e-4
)

100%|██████████| 150/150 [00:31<00:00,  4.76it/s]


#Задание 2
1. Обучите свой классификатор категории возраста
2. Попробуйте улучшить результат. Можно попробовать усложнить сетку, подвигать границы категорий, поискать новые данные, что угодно, кроме учиться на тесте :)
3. Какой подход оказался самым эффективным? Как думаете, почему?
4. Как считаете, где можно было бы применить такой классификатор в качестве вспомогательной задачи?


In [18]:
print(device)

cpu


In [19]:
import torch as t
from sklearn.metrics import f1_score

for i in tqdm(range(200)):

    optimizer.zero_grad()
    
    input, target = trainset.next_batch(BATCH_SIZE, device=device)
    out = model(input)
    
    loss = model.loss(out, target.type(torch.LongTensor))
    loss.backward()
    optimizer.step()

    if i % 50 == 0:
        model.eval()

        with torch.no_grad():
            optimizer.zero_grad()

            input, target = validset.next_batch(BATCH_SIZE, device=device)
            out = model(input)
            valid_loss = model.loss(out, target.type(torch.LongTensor))
            out, target = out.cpu().argmax(1).detach().numpy(), target.cpu().detach().numpy()
            
            print(f'f1_score:{f1_score(target,out,average=None)}')
            print("i {}, valid {}".format(i, valid_loss.item()))
            print("_________")

        model.train()

    if i % 50 == 0 and best_loss > valid_loss.item():
        best_loss = valid_loss.item()
        cnt = 0
    else:
        cnt += 1

    if cnt > patience:
        break
print('training finished')

  0%|          | 1/200 [00:03<11:16,  3.40s/it]

f1_score:[0.         0.70707071 0.        ]
i 0, valid 50.966033935546875
_________


 26%|██▌       | 51/200 [01:48<06:05,  2.45s/it]

f1_score:[0.         0.62365591 0.        ]
i 50, valid 1.0663971900939941
_________


 50%|█████     | 101/200 [03:33<04:02,  2.45s/it]

f1_score:[0.22222222 0.60240964 0.11111111]
i 100, valid 1.1418808698654175
_________


 76%|███████▌  | 151/200 [05:19<02:00,  2.46s/it]

f1_score:[0.32258065 0.65909091 0.        ]
i 150, valid 0.9994719624519348
_________


100%|██████████| 200/200 [07:00<00:00,  2.10s/it]

training finished


In [20]:
model.eval()
input,target=testset.next_batch(BATCH_SIZE,device=device)
out = model(input)
out, target = out.cpu().argmax(1).detach().numpy(), target.cpu().detach().numpy()


print(f1_score(target,out,average=None))
print(f1_score(target,out,average="weighted"))

[0.51282051 0.69333333 0.28571429]
0.5675915750915752


Вывод: Использован оптимайзер AdamW, CrossEntropyLoss, добавлен 1 Conv2d и 2 fc-слоя, немного измененены границы категорий, чтобы примерно сбалансировать классы. Как подзадача может использоваться в голосовой авторизации пользователей и т.д.